# Example of DOV search methods for soil data (bodemgegevens)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fsearch_bodem.ipynb)

## Use cases explained below
* Introduction to the bodem-objects
* Get bodemsites in a bounding box
* Get bodemlocaties with specific properties
* Get all direct and indirect bodemobservaties linked to a bodemlocatie
* Find all bodemlocaties where observations exist for organic carbon percentage in East-Flanders between 0 and 30 cm deep
* Calculate carbon stock in Ghent in the layer 0 - 23 cm

In [1]:
%matplotlib inline

import inspect, sys
import warnings; warnings.simplefilter('ignore')

In [2]:
import os
os.environ['PYDOV_BASE_URL'] = 'https://oefen.dov.vlaanderen.be/'

import pandas as pd

# check pydov path
import pydov

## Get information about the datatype 'Bodemlocatie'

Other datatypes are also possible:
* Bodemsite: BodemsiteSearch
* Bodemobservatie: BodemobservatieSearch

In [ ]:
from pydov.search.bodemlocatie import BodemlocatieSearch
bodemlocatie = BodemlocatieSearch()

A description is provided for the 'Bodemlocatie' datatype:

In [ ]:
bodemlocatie.get_description()

The different fields that are available for objects of the 'Bodemlocatie' datatype can be requested with the get_fields() method:

In [ ]:
fields = bodemlocatie.get_fields()

# print available fields
for f in fields.values():
    print(f['name'])

Alternatively, you can list all the fields and their details by inspecting the `get_fields()` output or the search instance itself in a notebook:

In [ ]:
bodemlocatie

## Example use cases

### Get bodemsites in a bounding box

Get data for all the bodemsites that are geographically located completely within the bounds of the specified box.

The coordinates are in the Belgian Lambert72 (EPSG:31370) coordinate system and are given in the order of lower left x, lower left y, upper right x, upper right y.

The same methods can be used for other bodem objects.

In [ ]:
from pydov.search.bodemsite import BodemsiteSearch
bodemsite = BodemsiteSearch()

In [ ]:
from pydov.util.location import Within, Box

df = bodemsite.search(location=Within(Box(148000, 160800, 160000, 169500, epsg=31370)))
df.head()

The dataframe contains a list of bodemsites. The available data are flattened to represent unique attributes per row of the dataframe.

Using the *pkey_bodemsite* field one can request the details of this bodemsite in a webbrowser:

In [ ]:
for pkey_bodemsite in set(df.pkey_bodemsite.head()):
    print(pkey_bodemsite)

### Get bodemlocaties with specific properties

Next to querying bodem objects based on their geographic location within a bounding box, we can also search for bodem objects matching a specific set of properties. 
The same methods can be used for all bodem objects.
For this we can build a query using a combination of the 'Bodemlocatie' fields and operators provided by the WFS protocol.

A list of possible operators can be found below:

In [ ]:
[i for i,j in inspect.getmembers(sys.modules['owslib.fes2'], inspect.isclass) if 'Property' in i]

In this example we build a query using the *PropertyIsEqualTo* operator to find all bodemlocaties with bodemstreek 'zandstreek'.
We use *max_features=10* to limit the results to 10.

In [ ]:
from owslib.fes2 import PropertyIsEqualTo

query = PropertyIsEqualTo(propertyname='bodemstreek',
                          literal='Zandstreek')
df = bodemlocatie.search(query=query, max_features=10)

df.head()

Once again we can use the *pkey_bodemlocatie* as a permanent link to the information of these bodemlocaties:

In [ ]:
for pkey_bodemlocatie in set(df.pkey_bodemlocatie.head()):
    print(pkey_bodemlocatie)

### Get all observaties linked to bodemlocatie

Get all bodemobservaties in a specific bodemlocatie.
Direct means bodemobservaties directly linked with a bodemlocatie.
Indirect means bodemobservaties linked with child-objects of the bodemlocatie, like diepteintervallen or monsters.

In [ ]:
from pydov.search.observatie import ObservatieSearch
from pydov.search.monster import MonsterSearch

from pydov.search.bodemlocatie import BodemlocatieSearch
from pydov.search.bodemdiepteinterval import BodemdiepteintervalSearch

from owslib.fes2 import PropertyIsEqualTo
from pydov.util.query import Join, FuzzyJoin

bodemlocatie = BodemlocatieSearch()
bodemdiepteinterval = BodemdiepteintervalSearch()
monster = MonsterSearch()
observatie = ObservatieSearch()

# prepare an empty dataframe to store all observations:
df_observaties = pd.DataFrame()

First, we get the soil locations (bodemlocaties) of our interest:

In [ ]:
df_bodemlocaties = bodemlocatie.search(query=PropertyIsEqualTo(propertyname='naam', literal='VMM_INF_52'), 
                                   return_fields=('pkey_bodemlocatie',))
df_bodemlocaties.head()

First, let's search for all observations that are linked directly to the soil location:

In [ ]:
# find observations linked directly to the soil location
df_obs = observatie.search(
    query=Join(df_bodemlocaties, on='pkey_parent', using='pkey_bodemlocatie')
)

# in this case, the parent is the soil location
df_obs['pkey_bodemlocatie'] = df_obs['pkey_parent']

# add these observations to the dataframe with observations
df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)
df_observaties

Observations can also be linked to a specific depth interval of the soil location. First, we have to find all depth intervals, and then search for observations linked to those:

In [ ]:
# find all depth intervals linked to the soil location
df_diepteintervallen = bodemdiepteinterval.search(
    query=Join(df_bodemlocaties, on='pkey_bodemlocatie'),
    return_fields=('pkey_diepteinterval','pkey_bodemlocatie',)
)

if len(df_diepteintervallen) > 0:

    # find all observations linked to the depth intervals
    df_obs = observatie.search(query=Join(df_diepteintervallen, on='pkey_parent', using='pkey_diepteinterval'))

    # merge the result with the depth intervals, to be able to link the observations with the soil location
    df_obs = df_obs.merge(
        df_diepteintervallen, left_on='pkey_parent', right_on='pkey_diepteinterval'
    ).drop(
        columns=('pkey_diepteinterval')
    )

    if len(df_obs) > 0:
        df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)

df_observaties

Observations can also be linked to a sample taken from the soil location. First we have to find all samples, and then find observations linked to those:

In [ ]:
# find all samples linked to the soil location
df_monsters = monster.search(
    query=FuzzyJoin(df_bodemlocaties, on='pkey_parents', using='pkey_bodemlocatie'),
    return_fields=('pkey_monster', 'pkey_parents')
)

# take the first soil location from the sample's parents as the linked soil location
df_monsters['pkey_bodemlocatie'] = df_monsters['pkey_parents'].apply(lambda x: [i for i in x if 'bodemlocatie' in i][0])

if len(df_monsters) > 0:

    # find all observations linked to samples
    df_obs = observatie.search(query=Join(df_monsters, on='pkey_parent', using='pkey_monster'))

    # merge the result with the samples, to be able to link them to their soil location
    df_obs = df_obs.merge(
        df_monsters, left_on='pkey_parent', right_on='pkey_monster'
    ).drop(
        columns=['pkey_parents', 'pkey_monster']
    )

    if len(df_obs) > 0:
        df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)

df_observaties

Lastly, observations can also be linked to a sample linked to a specific depth interval. First we have to find all samples linked to a depth interval, and then find observations linked to those:

In [ ]:
# find all samples linked to a depth interval
df_monsters_diepteintervallen = monster.search(
    query=FuzzyJoin(df_diepteintervallen, on='pkey_parents', using='pkey_diepteinterval'),
    return_fields=('pkey_monster', 'pkey_parents')
)

# take the first depth interval from the sample's parents as the linked depth interval
df_monsters_diepteintervallen['pkey_diepteinterval'] = df_monsters_diepteintervallen['pkey_parents'].apply(lambda x: [i for i in x if 'diepteinterval' in i][0])

# merge the result with the depth interval to be able to link them to the soil location
df_monsters_diepteintervallen = df_monsters_diepteintervallen.merge(
    df_diepteintervallen, on='pkey_diepteinterval'
).drop(
    columns=('pkey_diepteinterval')
)

if len(df_monsters_diepteintervallen) > 0:

    # find all observations linked to the samples
    df_obs = observatie.search(query=Join(df_monsters_diepteintervallen, on='pkey_parent', using='pkey_monster'))

    # merge the result with the previous result, to be able to link them to the soil location
    df_obs = df_obs.merge(
        df_monsters_diepteintervallen, left_on='pkey_parent', right_on='pkey_monster'
    ).drop(
        columns=['pkey_parents', 'pkey_monster']
    )

    if len(df_obs) > 0:
        df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)

df_observaties

We can also group these queries into a function, which will allow us to easily reuse this with any given dataframe of soil locations (containing at least the column `pkey_bodemlocatie`):

In [ ]:
def search_all_observations(df_bodemlocaties):
    df_observaties = pd.DataFrame()

    ## Direct observations

    # find observations linked directly to the soil location
    df_obs = observatie.search(
        query=Join(df_bodemlocaties, on='pkey_parent', using='pkey_bodemlocatie')
    )

    # in this case, the parent is the soil location
    df_obs['pkey_bodemlocatie'] = df_obs['pkey_parent']

    # add these observations to the dataframe with observations
    df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)


    ## Observations linked to depth intervals

    # find all depth intervals linked to the soil location
    df_diepteintervallen = bodemdiepteinterval.search(
        query=Join(df_bodemlocaties, on='pkey_bodemlocatie'),
        return_fields=('pkey_diepteinterval','pkey_bodemlocatie',)
    )

    if len(df_diepteintervallen) > 0:

        # find all observations linked to the depth intervals
        df_obs = observatie.search(query=Join(df_diepteintervallen, on='pkey_parent', using='pkey_diepteinterval'))

        # merge the result with the depth intervals, to be able to link the observations with the soil location
        df_obs = df_obs.merge(
            df_diepteintervallen, left_on='pkey_parent', right_on='pkey_diepteinterval'
        ).drop(
            columns=('pkey_diepteinterval')
        )

        if len(df_obs) > 0:
            df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)


    ## Observations linked to samples

    # find all samples linked to the soil location
    df_monsters = monster.search(
        query=FuzzyJoin(df_bodemlocaties, on='pkey_parents', using='pkey_bodemlocatie'),
        return_fields=('pkey_monster', 'pkey_parents')
    )

    # take the first soil location from the sample's parents as the linked soil location
    df_monsters['pkey_bodemlocatie'] = df_monsters['pkey_parents'].apply(lambda x: [i for i in x if 'bodemlocatie' in i][0])

    if len(df_monsters) > 0:

        # find all observations linked to samples
        df_obs = observatie.search(query=Join(df_monsters, on='pkey_parent', using='pkey_monster'))

        # merge the result with the samples, to be able to link them to their soil location
        df_obs = df_obs.merge(
            df_monsters, left_on='pkey_parent', right_on='pkey_monster'
        ).drop(
            columns=['pkey_parents', 'pkey_monster']
        )

        if len(df_obs) > 0:
            df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)


    ## Observations linked to samples linked to depth intervals

    # find all samples linked to a depth interval
    df_monsters_diepteintervallen = monster.search(
        query=FuzzyJoin(df_diepteintervallen, on='pkey_parents', using='pkey_diepteinterval'),
        return_fields=('pkey_monster', 'pkey_parents')
    )

    # take the first depth interval from the sample's parents as the linked depth interval
    df_monsters_diepteintervallen['pkey_diepteinterval'] = df_monsters_diepteintervallen['pkey_parents'].apply(lambda x: [i for i in x if 'diepteinterval' in i][0])

    # merge the result with the depth interval to be able to link them to the soil location
    df_monsters_diepteintervallen = df_monsters_diepteintervallen.merge(
        df_diepteintervallen, on='pkey_diepteinterval'
    ).drop(
        columns=('pkey_diepteinterval')
    )

    if len(df_monsters_diepteintervallen) > 0:

        # find all observations linked to the samples
        df_obs = observatie.search(query=Join(df_monsters_diepteintervallen, on='pkey_parent', using='pkey_monster'))

        # merge the result with the previous result, to be able to link them to the soil location
        df_obs = df_obs.merge(
            df_monsters_diepteintervallen, left_on='pkey_parent', right_on='pkey_monster'
        ).drop(
            columns=['pkey_parents', 'pkey_monster']
        )

        if len(df_obs) > 0:
            df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)

    return df_observaties

search_all_observations(df_bodemlocaties)

### Find all soil locations with a given soil classification

Get all soil locations with a given soil classification:

In [ ]:
from owslib.fes2 import PropertyIsEqualTo
from pydov.util.query import Join

from pydov.search.bodemclassificatie import BodemclassificatieSearch
from pydov.search.bodemlocatie import BodemlocatieSearch

bodemclassificatie = BodemclassificatieSearch()
bl_Scbz = bodemclassificatie.search(query=PropertyIsEqualTo('bodemtype', 'Scbz'), return_fields=['pkey_bodemlocatie'])

bodemlocatie = BodemlocatieSearch()
bl = bodemlocatie.search(query=Join(bl_Scbz, 'pkey_bodemlocatie'))
bl.head()

We can also get their observations, using the function we defined earlier:

In [ ]:
obs = search_all_observations(bl)
obs.head()

### Get all depth intervals and observations from a soil location

In [ ]:
from pydov.search.bodemlocatie import BodemlocatieSearch
from pydov.search.bodemdiepteinterval import BodemdiepteintervalSearch
from pydov.util.query import Join
from owslib.fes2 import PropertyIsEqualTo

bodemlocatie = BodemlocatieSearch()
bodemdiepteinterval = BodemdiepteintervalSearch()

bodemlocaties = bodemlocatie.search(query=PropertyIsEqualTo(propertyname='naam', literal='VMM_INF_52'),
                                    return_fields=('pkey_bodemlocatie',))

bodemdiepteintervallen = bodemdiepteinterval.search(
    query=Join(bodemlocaties, on='pkey_bodemlocatie'))
bodemdiepteintervallen

And get their observations:

In [ ]:
# direct observaties linked to diepteintervallen
df_observaties = observatie.search(query=Join(bodemdiepteintervallen, on='pkey_parent', using='pkey_diepteinterval'))
df_observaties.head()

### Find all bodemlocaties where observations exist for organic carbon percentage in Geraardsbergen between 0 and 30 cm deep

Get boundaries of Geraardsbergen by using a WFS

In [3]:
from owslib.etree import etree
from owslib.wfs import WebFeatureService
from pydov.util.location import (
    GmlFilter,
    Within,
)

from owslib.fes import PropertyIsEqualTo

gemeentegrenzen = WebFeatureService(
    'https://geo.api.vlaanderen.be/VRBG/wfs',
    version='1.1.0')

gemeente_filter = PropertyIsEqualTo(propertyname='NAAM', literal='Geraardsbergen')
gemeente_poly = gemeentegrenzen.getfeature(
    typename='VRBG:Refgem',
    filter=etree.tostring(gemeente_filter.toXML()).decode("utf8"),
    outputFormat='text/xml; subtype=gml/3.2').read()

Get bodemobservaties in Geraardsbergen with the requested properties:

In [5]:
import pandas as pd

from owslib.fes2 import PropertyIsEqualTo, PropertyIsGreaterThanOrEqualTo, PropertyIsLessThanOrEqualTo
from owslib.fes2 import And

from pydov.util.query import Join, FuzzyJoin

from pydov.search.bodemlocatie import BodemlocatieSearch
from pydov.search.bodemdiepteinterval import BodemdiepteintervalSearch
from pydov.search.observatie import ObservatieSearch
from pydov.search.monster import MonsterSearch

bodemlocatie = BodemlocatieSearch()
diepteinterval = BodemdiepteintervalSearch()
observatie = ObservatieSearch()
monster = MonsterSearch()

df_observaties = pd.DataFrame()

df_bodemlocaties = bodemlocatie.search(
    location=GmlFilter(gemeente_poly, Within)
)

parameter_name = 'Organische C - percentage (organische_c_perc)'

## Direct observations

# find observations linked directly to the soil location
df_obs = observatie.search(
    query=And([
        Join(df_bodemlocaties, on='pkey_parent', using='pkey_bodemlocatie'),
        PropertyIsEqualTo('parameter', parameter_name),
        PropertyIsGreaterThanOrEqualTo('diepte_van_m', '0'),
        PropertyIsLessThanOrEqualTo('diepte_tot_m', '0.3')
    ])
)

# in this case, the parent is the soil location
df_obs['pkey_bodemlocatie'] = df_obs['pkey_parent']

# add these observations to the dataframe with observations
df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)

## Observations linked to depth intervals

df_diepteintervallen = diepteinterval.search(
    query=And([
        Join(df_bodemlocaties, on='pkey_bodemlocatie'),
        PropertyIsGreaterThanOrEqualTo('bovengrens1_cm', '0'),
        PropertyIsLessThanOrEqualTo('ondergrens1_cm', '30')
    ]),
    return_fields=('pkey_diepteinterval', 'pkey_bodemlocatie')
)

if len(df_diepteintervallen) > 0:

    # find all observations linked to the depth intervals
    df_obs = observatie.search(
        query=And([
            Join(df_diepteintervallen, on='pkey_parent', using='pkey_diepteinterval'),
            PropertyIsEqualTo('parameter', parameter_name)
        ])
    )

    # merge the result with the depth intervals, to be able to link the observations with the soil location
    df_obs = df_obs.merge(
        df_diepteintervallen, left_on='pkey_parent', right_on='pkey_diepteinterval'
    ).drop(
        columns=('pkey_diepteinterval')
    )

    if len(df_obs) > 0:
        df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)


## Observations linked to samples

df_monsters = monster.search(
    query=And([
         FuzzyJoin(df_bodemlocaties, on='pkey_parents', using='pkey_bodemlocatie'),
         PropertyIsGreaterThanOrEqualTo(propertyname="diepte_van_m", literal = '0'),
         PropertyIsLessThanOrEqualTo(propertyname="diepte_tot_m", literal = '0.30')
    ]),
    return_fields=('pkey_monster', 'pkey_parents')
)

# take the first soil location from the sample's parents as the linked soil location
df_monsters['pkey_bodemlocatie'] = df_monsters['pkey_parents'].apply(lambda x: [i for i in x if 'bodemlocatie' in i][0])

if len(df_monsters) > 0:

    # find all observations linked to samples
    df_obs = observatie.search(
        query=And([
            Join(df_monsters, on='pkey_parent', using='pkey_monster'),
            PropertyIsEqualTo('parameter', parameter_name)
        ])
    )

    # merge the result with the samples, to be able to link them to their soil location
    df_obs = df_obs.merge(
        df_monsters, left_on='pkey_parent', right_on='pkey_monster'
    ).drop(
        columns=['pkey_parents', 'pkey_monster']
    )

    if len(df_obs) > 0:
        df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)


## Observations linked to samples linked to depth intervals

# find all samples linked to a depth interval
df_monsters_diepteintervallen = monster.search(
    query=FuzzyJoin(df_diepteintervallen, on='pkey_parents', using='pkey_diepteinterval'),
    return_fields=('pkey_monster', 'pkey_parents')
)

# take the first depth interval from the sample's parents as the linked depth interval
df_monsters_diepteintervallen['pkey_diepteinterval'] = df_monsters_diepteintervallen['pkey_parents'].apply(lambda x: [i for i in x if 'diepteinterval' in i][0])

# merge the result with the depth interval to be able to link them to the soil location
df_monsters_diepteintervallen = df_monsters_diepteintervallen.merge(
    df_diepteintervallen, on='pkey_diepteinterval'
).drop(
    columns=('pkey_diepteinterval')
)


if len(df_monsters_diepteintervallen) > 0:

    # find all observations linked to the samples
    df_obs = observatie.search(query=And([
        Join(df_monsters_diepteintervallen, on='pkey_parent', using='pkey_monster'),
        PropertyIsEqualTo('parameter', parameter_name)
    ]))

    # merge the result with the previous result, to be able to link them to the soil location
    df_obs = df_obs.merge(
        df_monsters_diepteintervallen, left_on='pkey_parent', right_on='pkey_monster'
    ).drop(
        columns=['pkey_parents', 'pkey_monster']
    )

    if len(df_obs) > 0:
        df_observaties = pd.concat([df_observaties, df_obs]).reset_index(drop=True)

df_observaties.head()


[000/001] .
[000/318] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/318] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/318] cccccccccccccccccccccccccccccccccccccccccccccccccc
[150/318] cccccccccccccccccccccccccccccccccccccccccccccccccc
[200/318] cccccccccccccccccccccccccccccccccccccccccccccccccc
[250/318] cccccccccccccccccccccccccccccccccccccccccccccccccc
[300/318] cccccccccccccccccc
[000/000] .[000/001] .
[000/000] .[000/001] .
[000/001] .
[000/001] .
[000/001] .


,pkey_observatie,pkey_parent,fenomeentijd,diepte_van_m,diepte_tot_m,parametergroep,parameter,detectieconditie,resultaat,eenheid,methode,uitvoerder,herkomst,pkey_bodemlocatie
0,https://oefen.dov.vlaanderen.be/data/observati...,https://oefen.dov.vlaanderen.be/data/monster/1...,1971-01-01,NaN,NaN,Bodem_chemisch,Organische C - percentage (organische_c_perc),NaN,0.52,%,Aardewerk nieuwe methode organische koolstof,Centrum voor Grondonderzoek (C.V.G.),LABO,https://oefen.dov.vlaanderen.be/data/bodemloca...
1,https://oefen.dov.vlaanderen.be/data/observati...,https://oefen.dov.vlaanderen.be/data/monster/1...,1971-01-01,NaN,NaN,Bodem_chemisch,Organische C - percentage (organische_c_perc),NaN,1.77,%,Aardewerk nieuwe methode organische koolstof,Centrum voor Grondonderzoek (C.V.G.),LABO,https://oefen.dov.vlaanderen.be/data/bodemloca...
2,https://oefen.dov.vlaanderen.be/data/observati...,https://oefen.dov.vlaanderen.be/data/monster/1...,1971-01-01,NaN,NaN,Bodem_chemisch,Organische C - percentage (organische_c_perc),NaN,0.57,%,Aardewerk nieuwe methode organische koolstof,Centrum voor Grondonderzoek (C.V.G.),LABO,https://oefen.dov.vlaanderen.be/data/bodemloca...
3,https://oefen.dov.vlaanderen.be/data/observati...,https://oefen.dov.vlaanderen.be/data/monster/1...,1971-01-01,NaN,NaN,Bodem_chemisch,Organische C - percentage (organische_c_perc),NaN,0.82,%,Aardewerk nieuwe methode organische koolstof,Centrum voor Grondonderzoek (C.V.G.),LABO,https://oefen.dov.vlaanderen.be/data/bodemloca...
4,https://oefen.dov.vlaanderen.be/data/observati...,https://oefen.dov.vlaanderen.be/data/monster/1...,1971-01-01,NaN,NaN,Bodem_chemisch,Organische C - percentage (organische_c_perc),NaN,0.70,%,Aardewerk nieuwe methode organische koolstof,Centrum voor Grondonderzoek (C.V.G.),LABO,https://oefen.dov.vlaanderen.be/data/bodemloca...


Now we have all observations with the requested properties. 
Next we need to link them with the bodemlocatie

In [6]:
import pandas as pd

# Merge the bodemlocatie information together with the observation information
merged = pd.merge(df_observaties, df_bodemlocaties, on="pkey_bodemlocatie", how='left')
merged.head()

,pkey_observatie,pkey_parent,fenomeentijd,diepte_van_m,diepte_tot_m,parametergroep,parameter,detectieconditie,resultaat,eenheid,...,type,waarnemingsdatum,doel,x,y,mv_mtaw,erfgoed,bodemstreek,invoerdatum,educatieve_waarde
0,https://oefen.dov.vlaanderen.be/data/observati...,https://oefen.dov.vlaanderen.be/data/monster/1...,1971-01-01,NaN,NaN,Bodem_chemisch,Organische C - percentage (organische_c_perc),NaN,0.52,%,...,boring,1971-01-01,bodemprofielen en oppervlaktemonsters karterin...,116565.0,159953.0,-9999.0,False,Leemstreek,2019-10-12,OK
1,https://oefen.dov.vlaanderen.be/data/observati...,https://oefen.dov.vlaanderen.be/data/monster/1...,1971-01-01,NaN,NaN,Bodem_chemisch,Organische C - percentage (organische_c_perc),NaN,1.77,%,...,boring,1971-01-01,bodemprofielen en oppervlaktemonsters karterin...,117207.0,164589.0,-9999.0,False,Zandleemstreek,2019-10-12,OK
2,https://oefen.dov.vlaanderen.be/data/observati...,https://oefen.dov.vlaanderen.be/data/monster/1...,1971-01-01,NaN,NaN,Bodem_chemisch,Organische C - percentage (organische_c_perc),NaN,0.57,%,...,boring,1971-01-01,bodemprofielen en oppervlaktemonsters karterin...,116892.0,163646.0,-9999.0,False,Zandleemstreek,2019-10-12,OK
3,https://oefen.dov.vlaanderen.be/data/observati...,https://oefen.dov.vlaanderen.be/data/monster/1...,1971-01-01,NaN,NaN,Bodem_chemisch,Organische C - percentage (organische_c_perc),NaN,0.82,%,...,boring,1971-01-01,bodemprofielen en oppervlaktemonsters karterin...,116985.0,165626.0,-9999.0,False,Zandleemstreek,2019-10-12,OK
4,https://oefen.dov.vlaanderen.be/data/observati...,https://oefen.dov.vlaanderen.be/data/monster/1...,1971-01-01,NaN,NaN,Bodem_chemisch,Organische C - percentage (organische_c_perc),NaN,0.70,%,...,boring,1971-01-01,bodemprofielen en oppervlaktemonsters karterin...,117722.0,165846.0,-9999.0,False,Zandleemstreek,2019-10-12,OK


To export the results to CSV, you can use for example: 
```python
merged.to_csv("test.csv")
```

We can plot also the results on a map
This can take some time!

In [9]:
import folium
from folium.plugins import MarkerCluster
from pyproj import Transformer

# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326", always_xy=True)
    x2,y2 = transformer.transform(x1, y1)
    return x2, y2

#convert coordinates to wgs84
merged['lon'], merged['lat'] = zip(*map(convert_latlon, merged['x'], merged['y']))

# Get only location and value
loclist = merged[['lat', 'lon']].values.tolist()

# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[merged['lat'].mean(), merged['lon'].mean()], zoom_start=10)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    popup = 'Bodemlocatie: ' + merged['pkey_bodemlocatie'][loc] 
    popup = popup + '<br> Observatie: ' + merged['pkey_observatie'][loc]
    popup = popup + '<br> Value: ' + merged['resultaat'][loc] + merged['eenheid'][loc]
    folium.Marker(loclist[loc], popup=popup).add_to(marker_cluster)
fmap

### Calculate carbon stock in Tervuren in the layer 0 - 23 cm

Get boundaries of Tervuren using WFS

In [ ]:
from owslib.etree import etree
from owslib.fes import PropertyIsEqualTo
from owslib.wfs import WebFeatureService
from pydov.util.location import (
    GmlFilter,
    Within,
)

stadsgrenzen = WebFeatureService(
    'https://geo.api.vlaanderen.be/VRBG/wfs',
    version='1.1.0')

gemeente_filter = PropertyIsEqualTo(propertyname='NAAM', literal='Tervuren')
gemeente_poly = stadsgrenzen.getfeature(
    typename='VRBG:Refgem',
    filter=etree.tostring(gemeente_filter.toXML()).decode("utf8"),
    outputFormat='text/xml; subtype=gml/3.2').read()


First get all observations in Tervuren for organisch C percentage in requested layer

In [ ]:
from owslib.fes2 import PropertyIsEqualTo, PropertyIsGreaterThan, PropertyIsLessThan
from owslib.fes2 import And

from pydov.search.bodemobservatie import BodemobservatieSearch

bodemobservatie = BodemobservatieSearch()

# all layers intersect the layer 0-23cm
carbon_observaties = bodemobservatie.search(
        location=GmlFilter(gemeente_poly, Within),
        query=And([
            PropertyIsEqualTo(propertyname="parameter", literal="Organische C - percentage"),
            PropertyIsGreaterThan(propertyname="diepte_tot_cm", literal = '0'),
            PropertyIsLessThan(propertyname="diepte_van_cm", literal = '23')
        ]),
        return_fields=('pkey_bodemlocatie', 'waarde'))
carbon_observaties = carbon_observaties.rename(columns={"waarde": "organic_c_percentage"})
carbon_observaties.head()


Then get all observations in Tervuren for bulkdensity in requested layer

In [ ]:
density_observaties = bodemobservatie.search(
        location=GmlFilter(gemeente_poly, Within),
        query=And([
            PropertyIsEqualTo(propertyname="parameter",
                              literal="Bulkdensiteit - gemeten"),
            PropertyIsGreaterThan(propertyname="diepte_tot_cm", literal = '0'),
            PropertyIsLessThan(propertyname="diepte_van_cm", literal = '23')
        ]),
        return_fields=('pkey_bodemlocatie', 'waarde'))

density_observaties = density_observaties.rename(columns={"waarde": "bulkdensity"})
density_observaties.head()

Merge results together based on their bodemlocatie. Only remains the records where both parameters exists

In [ ]:
import pandas as pd

merged = pd.merge(carbon_observaties, density_observaties, on="pkey_bodemlocatie")

merged.head()

### Filter Aardewerk soil locations
Since we know that Aardewerk soil locations make use of a specific suffix, a query could be built filtering these out.

Since we only need to match a partial string in the name, we will build a query using the *PropertyIsLike* operator to find all Aardewerk bodemlocaties.
We use *max_features=10* to limit the results to 10.

In [ ]:
from owslib.fes2 import PropertyIsLike

query = PropertyIsLike(propertyname='naam',
                       literal='KART_PROF_%', wildCard='%')
df = bodemlocatie.search(query=query, max_features=10)

df.head()

As seen in the soil data example, we can use the *pkey_bodemlocatie* as a permanent link to the information of these bodemlocaties:

In [ ]:
for pkey_bodemlocatie in set(df.pkey_bodemlocatie):
    print(pkey_bodemlocatie)